In [ ]:
# CELLA 1: Upload File Excel
from google.colab import files
print("📤 Carica il tuo file prompts.xlsx")
uploaded = files.upload()


📤 Carica il tuo file prompts.xlsx


Saving prompts.xlsx to prompts.xlsx


In [ ]:
# CELLA 2: Installazione dipendenze
print("⚙️ Installazione dipendenze in corso...")

# Installa browser e driver
!apt-get update -qq > /dev/null 2>&1
!apt-get install -y chromium-browser chromium-chromedriver > /dev/null 2>&1

# Installa librerie Python
!pip install -q playwright pandas openpyxl

# Installa browser Playwright
!playwright install chromium
!playwright install-deps

print("✅ Installazione completata!")

⚙️ Installazione dipendenze in corso...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 MB 21.1 MB/s eta 0:00:00
(node:1850) [DEP0169] DeprecationWarning: `url.parse()` behavior is not standardized and prone to errors that have security implications. Use the WHATWG URL API instead. CVEs are not issued for `url.parse()` vulnerabilities.
(Use `node --trace-deprecation ...` to show where the warning was created)
164.7 MiB [] 0% 0.0s164.7 MiB [] 0% 211.6s164.7 MiB [] 0% 227.0s164.7 MiB [] 0% 713.9s164.7 MiB [] 0% 598.6s164.7 MiB [] 0% 803.5s164.7 MiB [] 0% 711.6s164.7 MiB [] 0% 643.0s164.7 MiB [] 0% 594.1s164.7 MiB [] 0% 554.9s164.7 MiB [] 0% 551.0s164.7 MiB [] 0% 519.0s164.7 MiB [] 0% 495.9s164.7 MiB [] 0% 477.1s164.7 MiB [] 0% 460.3s164.7 MiB [] 0% 441.5s164.7 MiB [] 0% 426.4s164.7 MiB [] 0% 412.4s164.7 MiB [] 0% 394.0s164.7 MiB [] 0% 373.0s164.7 MiB [] 0% 348.6s164.7 MiB [] 0% 327.8s164.7 MiB [] 0% 310.6s164.7 MiB [] 0% 295.1s164.7 MiB [] 0% 282.1s164.7 MiB [] 0% 270.3s164.7 MiB [

In [ ]:
# CELLA 3: Configurazione credenziali Digen.ai
import getpass

print("🔐 CONFIGURAZIONE LOGIN DIGEN.AI")
print("="*60)
print("Inserisci le credenziali del tuo account Digen.ai")
print("(Se non hai account, creane uno su https://digen.ai)")
print("="*60)
print()

EMAIL = input("📧 Email: ")
PASSWORD = getpass.getpass("🔒 Password: ")

print("\n✅ Credenziali salvate!")


🔐 CONFIGURAZIONE LOGIN DIGEN.AI
Inserisci le credenziali del tuo account Digen.ai
(Se non hai account, creane uno su https://digen.ai)

📧 Email: soldiora@live.it
🔒 Password: ··········

✅ Credenziali salvate!


In [ ]:
# CELLA: Elimina screenshot vecchi
import shutil
from pathlib import Path

debug_folder = Path("debug_screenshots")

if debug_folder.exists():
    # Elimina cartella e tutto il contenuto
    shutil.rmtree(debug_folder)
    print(f"✅ Eliminati tutti gli screenshot da: {debug_folder}")

    # Ricrea cartella vuota
    debug_folder.mkdir()
    print(f"✅ Cartella debug ricreata vuota")
else:
    print(f"ℹ️ Cartella {debug_folder} non esiste")

# Verifica
screenshots = list(debug_folder.glob("*.png"))
print(f"\n📸 Screenshot rimasti: {len(screenshots)}")


✅ Eliminati tutti gli screenshot da: debug_screenshots
✅ Cartella debug ricreata vuota

📸 Screenshot rimasti: 0


In [ ]:
# CELLA 3.5: PULISCI TRACKING (usa solo se vuoi ricominciare da zero)
import json
from pathlib import Path

tracking_file = Path("video_tracking.json")

if tracking_file.exists():
    with open(tracking_file, 'r') as f:
        completed = json.load(f)

    print(f"📋 Video nel tracking: {len(completed)}")
    for v in completed:
        print(f"   • {v}")

    print("\n⚠️ Vuoi pulire il tracking? (ricomincerà da zero)")
    # Decommentare la riga sotto per pulire:
    # tracking_file.unlink()
    # print("✅ Tracking pulito!")
else:
    print("📋 Nessun tracking esistente")


📋 Video nel tracking: 3
   • 3_08
   • 3_07
   • 03_coffee_shop

⚠️ Vuoi pulire il tracking? (ricomincerà da zero)


In [ ]:
# CELLA 4: VERSIONE UNIVERSALE - Scegli Veo o Sora2

import pandas as pd
from playwright.async_api import async_playwright
import asyncio
import json
from pathlib import Path

# 🆕 CONFIGURAZIONE: Scegli il modello
MODEL = "sora2"  # Cambia in "veo" o "sora2"

async def automate_digen_universal():
    """
    UNIVERSALE: Funziona con Veo o Sora2
    """

    # URL basato sul modello
    if MODEL == "veo":
        BASE_URL = "https://digen.ai/veo?orientation=landscape"
        print("🎬 Modalità: VEO\n")
    elif MODEL == "sora2":
        BASE_URL = "https://digen.ai/sora?seconds=15&orientation=landscape"
        print("🎬 Modalità: SORA 2\n")
    else:
        print("❌ MODEL deve essere 'veo' o 'sora2'")
        return

    print("📂 Lettura file Excel...")
    df = pd.read_excel('prompts.xlsx')
    total_prompts = len(df)
    print(f"✅ Trovati {total_prompts} prompt\n")

    download_folder = Path("video_generati")
    download_folder.mkdir(exist_ok=True)
    debug_folder = Path("debug_screenshots")
    debug_folder.mkdir(exist_ok=True)

    cookies_file = Path("digen_cookies.json")
    tracking_file = Path(f"video_tracking_{MODEL}.json")  # Tracking separato per modello

    if tracking_file.exists():
        with open(tracking_file, 'r') as f:
            completed_prompts = set(json.load(f))
    else:
        completed_prompts = set()

    # [FUNZIONI - stesse di prima]
    async def insert_prompt_multiline(page, prompt_text):
        try:
            textarea = page.locator('textarea').first
            await textarea.click()
            await asyncio.sleep(0.3)
            await page.keyboard.press('Control+A')
            await page.keyboard.press('Backspace')
            await asyncio.sleep(0.3)
            await textarea.fill(prompt_text)
            await asyncio.sleep(0.5)
            current_value = await textarea.input_value()
            return len(current_value) >= len(prompt_text) * 0.9
        except:
            return False

    async def click_generate(page):
        try:
            textarea = page.locator('textarea').first
            textarea_box = await textarea.bounding_box()
            if not textarea_box:
                await page.keyboard.press('Control+Enter')
                return True, "Ctrl+Enter"
            textarea_y = textarea_box['y']
            all_buttons = await page.locator('button:visible').all()
            same_row = []
            for btn in all_buttons:
                try:
                    box = await btn.bounding_box()
                    if not box or abs(box['y'] - textarea_y) >= 50:
                        continue
                    html = await btn.inner_html()
                    text = await btn.inner_text()
                    if '+' not in text:
                        same_row.append((btn, box['x'], 'svg' in html.lower()))
                except:
                    continue
            if not same_row:
                await page.keyboard.press('Control+Enter')
                return True, "Ctrl+Enter"
            same_row.sort(key=lambda x: x[1], reverse=True)
            for btn, x, has_svg in same_row:
                if has_svg:
                    await btn.click()
                    await asyncio.sleep(2)
                    return True, "Generate"
            await same_row[0][0].click()
            await asyncio.sleep(2)
            return True, "Generate"
        except:
            try:
                await page.keyboard.press('Control+Enter')
                return True, "Ctrl+Enter"
            except:
                return False, "Errore"

    async def count_sidebar_videos(page):
        try:
            return await page.locator('.creation-item, .video-item, [class*="creation"], video').count()
        except:
            return 0

    async def wait_for_new_video(page, initial, timeout=600):
        start = asyncio.get_event_loop().time()
        while (asyncio.get_event_loop().time() - start) < timeout:
            await asyncio.sleep(5)
            current = await count_sidebar_videos(page)
            if current > initial:
                return True
            elapsed = int(asyncio.get_event_loop().time() - start)
            if elapsed % 30 == 0 and elapsed > 0:
                print(f"      ⏳ {elapsed}s...")
        return False

    async def wait_for_download_ready(page, timeout=120):
        start = asyncio.get_event_loop().time()
        while (asyncio.get_event_loop().time() - start) < timeout:
            try:
                all_buttons = await page.locator('button:visible').all()
                for btn in all_buttons:
                    try:
                        box = await btn.bounding_box()
                        if not box or box['x'] < 1100 or box['y'] > 500:
                            continue
                        is_disabled = await btn.get_attribute('disabled')
                        if is_disabled is None:
                            html = await btn.inner_html()
                            if 'svg' in html.lower() and box['width'] < 50:
                                return True
                    except:
                        continue
                await asyncio.sleep(3)
            except:
                await asyncio.sleep(3)
        return False

    async def download_latest_video(page, file_name, download_folder):
        try:
            await asyncio.sleep(2)
            if not await wait_for_download_ready(page):
                return False, "Download non pronto", None
            await asyncio.sleep(2)
            all_buttons = await page.locator('button:visible').all()
            candidates = []
            for btn in all_buttons:
                try:
                    box = await btn.bounding_box()
                    if not box or box['x'] < 1100:
                        continue
                    is_disabled = await btn.get_attribute('disabled')
                    if is_disabled is not None:
                        continue
                    html = await btn.inner_html()
                    if box['width'] < 50 and box['height'] < 50 and 'svg' in html.lower():
                        candidates.append((btn, box['x'], box['y']))
                except:
                    continue
            if not candidates:
                return False, "Nessun button", None
            candidates.sort(key=lambda x: x[2])
            first_y = candidates[0][2]
            same_row = [c for c in candidates if abs(c[2] - first_y) < 10]
            same_row.sort(key=lambda x: x[1], reverse=True)
            download_btn = same_row[0][0]
            async with page.expect_download(timeout=90000) as dl_info:
                await download_btn.click()
            download = await dl_info.value
            ext = Path(download.suggested_filename).suffix or '.mp4'
            final_path = download_folder / f"{file_name}{ext}"
            await download.save_as(str(final_path))
            size = final_path.stat().st_size
            if size < 10000:
                final_path.unlink()
                return False, "File corrotto", None
            size_mb = size / (1024 * 1024)
            return True, f"{size_mb:.1f} MB", final_path
        except Exception as e:
            return False, str(e), None

    # MAIN
    print("🌐 Avvio browser...")
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True, args=['--no-sandbox', '--disable-dev-shm-usage'])
        context = await browser.new_context(
            viewport={'width': 1920, 'height': 1080},
            accept_downloads=True,
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        )
        page = await context.new_page()
        page.on("dialog", lambda dialog: dialog.accept())

        try:
            # LOGIN
            if cookies_file.exists():
                print("📥 Carico sessione...")
                with open(cookies_file, 'r') as f:
                    cookies = json.load(f)
                await context.add_cookies(cookies)
                await page.goto(BASE_URL, timeout=30000)  # 🆕 USA BASE_URL
                await asyncio.sleep(3)
                print("✅ Sessione valida!\n")
            else:
                print("🔐 Login...")
                await page.goto("https://digen.ai/signin", timeout=30000)
                await asyncio.sleep(3)
                await page.fill('input[type="email"]', EMAIL)
                await page.fill('input[type="password"]', PASSWORD)
                await page.click('button[type="submit"]')
                await asyncio.sleep(6)
                cookies = await context.cookies()
                with open(cookies_file, 'w') as f:
                    json.dump(cookies, f)
                print("✅ Login OK!\n")
                await page.goto(BASE_URL, timeout=30000)  # 🆕 USA BASE_URL
                await asyncio.sleep(3)

            print(f"🎬 INIZIO GENERAZIONE ({MODEL.upper()})")
            print("="*60 + "\n")

            for index, row in df.iterrows():
                prompt_text = str(row['Prompt']).strip()
                file_name = str(row['NomeFile']).strip()

                print(f"📝 VIDEO {index + 1}/{total_prompts}: {file_name}")

                if file_name in completed_prompts:
                    print(f"   ⏭️ Già completato\n")
                    continue

                video_success = False

                for retry in range(2):
                    try:
                        if index > 0 or retry > 0:
                            print("🔄 Ricarico...")
                            await page.goto(BASE_URL, timeout=30000)  # 🆕 USA BASE_URL
                            await asyncio.sleep(4)

                        initial = await count_sidebar_videos(page)

                        print("✏️ Inserisco prompt...")
                        if not await insert_prompt_multiline(page, prompt_text):
                            if retry < 1:
                                continue
                            else:
                                break
                        print("   ✅ Inserito")

                        print("🚀 Generate...")
                        success, msg = await click_generate(page)
                        if not success:
                            if retry < 1:
                                continue
                            else:
                                break
                        print(f"   ✅ {msg}")
                        await asyncio.sleep(5)

                        print("⏳ Attendo...")
                        if not await wait_for_new_video(page, initial):
                            if retry < 1:
                                continue
                            else:
                                break
                        print("✅ Generato!")

                        print("📥 Scarico...")
                        success, msg, path = await download_latest_video(page, file_name, download_folder)

                        if success:
                            print(f"   💾 {msg}")
                            print(f"✅ VIDEO {index + 1}/{total_prompts} COMPLETATO!\n")
                            completed_prompts.add(file_name)
                            with open(tracking_file, 'w') as f:
                                json.dump(list(completed_prompts), f)
                            video_success = True
                            break
                        else:
                            print(f"   ⚠️ {msg}\n")
                            completed_prompts.add(file_name)
                            with open(tracking_file, 'w') as f:
                                json.dump(list(completed_prompts), f)
                            video_success = True
                            break

                    except Exception as e:
                        print(f"   ⚠️ {e}")
                        if retry < 1:
                            await asyncio.sleep(10)
                            continue

                if index < total_prompts - 1:
                    await asyncio.sleep(20)

            print("\n" + "="*60)
            print("🎉 COMPLETATO!")
            print("="*60)

            videos = sorted(download_folder.glob("*.mp4"))
            total_mb = sum(v.stat().st_size for v in videos) / (1024 * 1024)

            print(f"\n✅ Video: {len(videos)}/{total_prompts}")
            print(f"💾 Totale: {total_mb:.1f} MB\n")

        except Exception as e:
            print(f"\n❌ ERRORE: {e}")
        finally:
            await browser.close()

# ESEGUI
await automate_digen_universal()


🎬 Modalità: SORA 2

📂 Lettura file Excel...
✅ Trovati 2 prompt

🌐 Avvio browser...
📥 Carico sessione...
✅ Sessione valida!

🎬 INIZIO GENERAZIONE (SORA2)

📝 VIDEO 1/2: 3_07
✏️ Inserisco prompt...
   ✅ Inserito
🚀 Generate...
   ✅ Ctrl+Enter
⏳ Attendo...
      ⏳ 30s...
      ⏳ 60s...
      ⏳ 90s...
      ⏳ 600s...
🔄 Ricarico...
✏️ Inserisco prompt...
   ✅ Inserito
🚀 Generate...
   ✅ Ctrl+Enter
⏳ Attendo...
      ⏳ 30s...
      ⏳ 60s...
      ⏳ 90s...


CancelledError: 

In [ ]:
# CELLA SAVE: SALVA TUTTO SU GOOGLE DRIVE (esegui UNA VOLTA)

from google.colab import drive
drive.mount('/content/drive')

import shutil
from pathlib import Path

# Crea cartella progetto su Drive
project_folder = Path("/content/drive/MyDrive/VeoAutomation")
project_folder.mkdir(exist_ok=True)

print("📁 Creo struttura progetto su Google Drive...")

# Salva script Python
script_content = """
# SCRIPT VEO AUTOMATION
# Copia tutto il codice della CELLA 4 qui dentro
"""

with open(project_folder / "veo_automation.py", 'w') as f:
    f.write(script_content)

# Copia file importanti
files_to_save = [
    "prompts.xlsx",
    "digen_cookies.json",
    "video_tracking.json"
]

for file in files_to_save:
    if Path(file).exists():
        shutil.copy(file, project_folder / file)
        print(f"✅ Salvato: {file}")

print(f"\n🎉 Progetto salvato in: {project_folder}")
print("\n💡 Ogni giorno:")
print("1. Apri questo notebook")
print("2. Esegui: Runtime > Run all")
print("3. Aspetta che finisca")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📁 Creo struttura progetto su Google Drive...
✅ Salvato: prompts.xlsx
✅ Salvato: digen_cookies.json
✅ Salvato: video_tracking.json

🎉 Progetto salvato in: /content/drive/MyDrive/VeoAutomation

💡 Ogni giorno:
1. Apri questo notebook
2. Esegui: Runtime > Run all
3. Aspetta che finisca
